In [10]:
import torch
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Lambda, Compose, Normalize
from collections import defaultdict
from torch.utils.data import random_split
import copy

from NN_utils import *
from NN_utils.train_and_eval import *
from uncertainty import train_NN_with_g
from uncertainty.losses import penalized_uncertainty
import uncertainty.comparison as unc_comp
import uncertainty.quantifications as unc


from torch.utils.data import random_split

In [2]:
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

True


In [3]:
transforms_train = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
transforms_ = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

In [4]:
training_data = datasets.CIFAR100(
root="data",
 train=True,
 download=True,
transform=transforms_train)

test_data = datasets.CIFAR100(
root="data",
train=False,
download=True,
transform=transforms_)

train_size = int(0.5*len(training_data))
val_size = len(training_data) - train_size
training_data, validation_data = random_split(training_data, [train_size, val_size])

Files already downloaded and verified
Files already downloaded and verified


In [5]:
batch_size = 12
train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle = True)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size,shuffle = True)
test_dataloader = DataLoader(test_data, batch_size=100)

In [6]:

from NN_models.CIFAR100 import Model_CNN_100_with_g
model_100 = Model_CNN_100_with_g()
model_100.to(dev)

from NN_models import Model_CNN_with_g
model_90 = Model_CNN_with_g(90)
model_90.to(dev);

In [7]:
optimizer = torch.optim.SGD(model_90.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

Trainer_90 = Trainer(model_90, optimizer, loss_fn, print_loss=True, keep_hist=True)


In [9]:
optimizer = torch.optim.SGD(model_100.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

Trainer_100 = Trainer(model_100, optimizer, loss_fn, print_loss=True, keep_hist=True)

Trainer_100.fit(train_dataloader,80)
acc, g, bce = model_metrics(model_100,nn.NLLLoss(),train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_100,nn.NLLLoss(),test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

Epoch  1 , loss =  1.6456514187371662
Epoch  1 , loss =  1.6479998733455068
Epoch  1 , loss =  1.6521076676712843
Epoch  1 , loss =  1.6208714493863186
Epoch  1 , loss =  1.598949745383235
Epoch  1 , loss =  1.6087585327263758
Epoch  1 , loss =  1.6181315141881953
Epoch  1 , loss =  1.61206546096907
Epoch  1 , loss =  1.5939808384928273
Epoch  1 , loss =  1.594114494100642
Epoch  1 , loss =  1.5919451359480676
Epoch  1 , loss =  1.587409650867594
Epoch  1 , loss =  1.5828316981069415
Epoch  1 , loss =  1.5885717631988012
Epoch  1 , loss =  1.555963343656452
Epoch  1 , loss =  1.5611013166048706
Epoch  1 , loss =  1.5432939287167822
Epoch  1 , loss =  1.539942075560967
Epoch  1 , loss =  1.566072698003271
Epoch  1 , loss =  1.5501842398103505
Epoch  1 , loss =  1.5491820641877325
Epoch  1 , loss =  1.542850535684721
Epoch  1 , loss =  1.5352791089300002
Epoch  1 , loss =  1.5332851515450083
Epoch  1 , loss =  1.5239705643589803
Epoch  1 , loss =  1.520536091529019
Epoch  1 , loss =  1.5